In [21]:
import csv
import os

import numpy as np
import scipy.stats
import sklearn.decomposition
from matplotlib import pyplot as plt
from PIL import Image

![alt text](L2.png "Title")

In [22]:
RGB = ["r", "g", "b"]
fieldnames = [
    "mean_r", "mean_g", "mean_b",
    "var_r", "var_g", "var_b",
    "skewness_r", "skewness_g", "skewness_b",
    "kurtosis_r", "kurtosis_g", "kurtosis_b"
]
DECIMALS = 4
COMPONENTS = [10, 50, 100, 150, 250]

def metric_mse(C, S):
    return round(float(np.power(C - S, 2).sum()) / float((C.shape[0] * C.shape[1])), DECIMALS)

def pca_perform(image, components):
    pca = sklearn.decomposition.PCA(components, svd_solver="full")
    image_pca =pca.fit_transform(image)
    pca_image_inverse = pca.inverse_transform(image_pca)
    return pca_image_inverse.astype("int")

In [23]:
class Img(object):
    
    def __init__(self, array):
        self.img = array
    
    @property
    def im_mean(self):
        return [round(np.mean(self.img[:, :, ch]), DECIMALS) for ch in range(self.img.shape[-1])]

    @property
    def im_var(self):
        return [round(np.var(self.img[:, :, ch]), DECIMALS) for ch in range(self.img.shape[-1])]
    
    @property
    def im_skewness(self):
        return [round(scipy.stats.skew(self.img[:, :, ch], axis=None), DECIMALS) for ch in range(self.img.shape[-1])]
    
    @property
    def im_kurtosis(self):
        return [round(scipy.stats.kurtosis(self.img[:, :, ch], axis=None), DECIMALS) for ch in range(self.img.shape[-1])]
        

In [24]:
image_dir = "../JImages/"
filename = "im24820.jpg"
image = np.array(Image.open(f"{image_dir}{filename}"))
image.shape

(500, 333, 3)

In [25]:
img = Img(image)


def get_vectors(img):
    result = dict()
    for prop in dir(img):
        
        if prop.startswith("im_"):
            r_res, g_res, b_res = getattr(img, prop)
            result.update({f"{prop[3:]}_r": r_res, f"{prop[3:]}_g": g_res, f"{prop[3:]}_b": b_res})
    return result

get_vectors(img)

{'kurtosis_r': -1.4703,
 'kurtosis_g': -0.4337,
 'kurtosis_b': -0.7997,
 'mean_r': 114.9447,
 'mean_g': 109.9738,
 'mean_b': 124.5167,
 'skewness_r': 0.4544,
 'skewness_g': 0.7866,
 'skewness_b': 0.0547,
 'var_r': 7815.2901,
 'var_g': 4209.9408,
 'var_b': 3426.5151}

### 3. Form Vectors from Image statistical moments

In [26]:
vectors = [[],[],[],[],[],[],[],[],[],[],[],[]]

files = os.listdir(f"{image_dir}")
for i in range(len(files)):
    image = files[i]
    result = dict()
    img = Img(np.array(Image.open(f"{image_dir}{image}")))
    properties = get_vectors(img)
    
    for v in enumerate(fieldnames):
        
        vectors[v[0]].append(properties[v[1]])

ARRAY_4 = np.array(vectors)
del vectors
    
print(ARRAY_4)

means = np.round(np.mean(ARRAY_4[0:12], axis=1), 4)

[[ 66.6366  92.8637 210.4733 ... 105.4733 123.7653 180.9778]
 [ 53.3066  87.5544 173.8455 ... 105.4733 130.3493 179.6594]
 [ 64.5057  84.5643 197.4258 ... 105.4733 130.4133 180.641 ]
 ...
 [  1.8721   5.5533   1.2711 ...  -0.4432  -0.6858  -1.0347]
 [  4.2684   1.4616  -1.4136 ...  -0.4432  -0.5736  -1.0848]
 [ -0.6338   2.434   -0.5014 ...  -0.4432  -0.903   -1.0433]]


### 4. Form Models from Vectors

In [27]:
np.set_printoptions(suppress=True)

print(f"Vector 1 mean={means[0:3].shape} \ncov={np.round(np.cov(ARRAY_4[0:3]), 4).shape} (shape)\n")

print(f"Vector 2 mean={means[0:6].shape} \ncov={np.round(np.cov(ARRAY_4[0:6]), 4).shape} (shape)\n")

print(f"Vector 3 mean={means[0:9].shape} \ncov={np.round(np.cov(ARRAY_4[0:9]), 4).shape} (shape)\n")

print(f"Vector 4 mean={means[0:12]} \ncov={np.round(np.cov(ARRAY_4[0:12]), 4)}")

Vector 1 mean=(3,) 
cov=(3, 3) (shape)

Vector 2 mean=(6,) 
cov=(6, 6) (shape)

Vector 3 mean=(9,) 
cov=(9, 9) (shape)

Vector 4 mean=[ 119.3066  108.8693   99.3559 4558.6918 4074.3903 3965.6628    0.0939
    0.2372    0.4519    0.2847    0.3512    1.2199] 
cov=[[   1411.7679    1003.191      887.7191   10448.0588   21774.3271
    18920.7391     -30.5097     -22.9977     -18.8563      20.8141
       12.5593      -7.1094]
 [   1003.191     1145.7354    1278.6298    4513.7918   22512.8834
    34706.2517     -19.2905     -23.3732     -27.55         9.7395
       -1.1419     -45.9327]
 [    887.7191    1278.6298    1961.986    -3025.8933   23326.1939
    49024.5365     -14.8347     -24.0282     -41.6017      10.034
       -3.1848     -80.8021]
 [  10448.0588    4513.7918   -3025.8933 5436850.9942 4635915.1306
  3455324.0869     -69.3735      48.8271      56.8517   -3933.2016
    -3488.6763   -3440.2466]
 [  21774.3271   22512.8834   23326.1939 4635915.1306 5507455.7934
  4707084.6931    -2

### 5.1 Image color chanels decomposition

In [28]:
def pca_decomposition_mes(image):
    res = []
    max_components = min(image.shape)
    for c in COMPONENTS:
        image_pca = pca_perform(image, c)
        mse = metric_mse(image, image_pca)
        res.append(mse)
    image_pca = pca_perform(image, max_components)
    mse = metric_mse(image, image_pca)
    res.append(mse)
    return res

In [29]:
image = np.array(Image.open(f"{image_dir}{filename}"))
res = pca_decomposition_mes(image[:, :, 0])
print(res)

[580.6245, 108.4055, 37.1901, 14.4189, 1.5187, 0.4562]


In [ ]:
R = [[], [], [], [], [], []]
G = [[], [], [], [], [], []]
B = [[], [], [], [], [], []]
files = os.listdir(f"{image_dir}")
for i in range(len(files)):
    img = files[i]
    image = np.array(Image.open(f"{image_dir}{img}"))
    res = pca_decomposition_mes(image[:, :, 0])
    for r in range(len(res)):
        R[r].append(res[r])

    res = pca_decomposition_mes(image[:, :, 1])
    for r in range(len(res)):
        G[r].append(res[r])

    res = pca_decomposition_mes(image[:, :, 2])
    for r in range(len(res)):
        B[r].append(res[r])

### 5.2 Plot

In [ ]:
plt.figure(figsize=(10,6))
figure = plt.subplot()


for i in zip([R, G, B], ["r", "g-.", "b--"], [1, 2, 3]):
    plt.subplot(f"13{i[2]}")
    arr = np.array(i[0])
    mean_mse = np.round(np.mean(arr, axis=1), DECIMALS)
    plt.plot([*COMPONENTS, 300], mean_mse, i[1], lw=3.0)
    plt.grid(b=True, which='major', color='k', linestyle='--')
    plt.ylim((0, 850))
    plt.xlim((0, 320))


components = [i for i in zip([*COMPONENTS, 332], mean_mse)]
for c in components:
    print(f"Number components: {c[0]}; MSE: {c[1]}")
plt.savefig("l2res.png")
plt.show()

### 6.1 Make explicit Markov matrix of 1 and 2 order

In [ ]:
image = np.array(Image.open(f"{image_dir}{filename}"))
image.shape

In [ ]:
np.warnings.filterwarnings('ignore')
shape = (256, 256)
R = np.zeros(shape)
G = np.zeros(shape)
B = np.zeros(shape)
c = 0
for i in image:
    previous = i[0]
    for j in range(1,len(i)):
        R[previous[0]][i[j][0]] += 1
        G[previous[1]][i[j][1]] += 1
        B[previous[2]][i[j][2]] += 1

for vi in enumerate((R, G, B)):
    i = vi[0]
    v = vi[1]
    row_sums = v.sum(axis=1)
    assert int(np.sum(row_sums)) == image.shape[0] * (image.shape[1] - 1)
    matrix_normalized = v / row_sums[:, np.newaxis]
    np.nan_to_num(matrix_normalized, False)
    np.savetxt(f"markov_{('R', 'G', 'B')[i]}_1.txt", matrix_normalized, fmt="%.5f", delimiter="\t")
    matrix_norm_2 = np.dot(matrix_normalized, matrix_normalized)
    np.savetxt(f"markov_{('R', 'G', 'B')[i]}_2.txt", matrix_norm_2, fmt="%.5f", delimiter="\t")

In [ ]:
a = 0
for i in range(len(matrix_normalized)):
    for j in range(len(matrix_normalized[i])):
        if i == j:
            a += 1

In [ ]:
matrix_norm_5 = np.dot(np.dot(np.dot(np.dot(matrix_normalized, matrix_normalized), matrix_normalized), matrix_normalized), matrix_normalized)

In [ ]:
a = 0
mn = np.zeros((1, 256))
print(matrix_norm_5)
for i in range(len(matrix_norm_5)):
    for j in range(len(matrix_norm_5[i])):
        if i == j:
            print(f"{np.round(matrix_norm_5[i][j], 6)}")
            a += 1

np.savetxt("diag5.txt", matrix_norm_5,   fmt="%.6f", delimiter="\t")            
np.savetxt("matrix5.txt", matrix_norm_5,   fmt="%.6f", delimiter="\t")